Tabu Search Ketapang

In [1]:
from ts import *

road_points = {
    0: (-1.064124, 110.685258),   
    1: (-1.919500, 110.671412),  
    2: (-0.625036, 110.427354)    
}

routing_ts = ClusterBasedDroneRoutingTS(
    csv_file="ketapang-12-08-24_merged_clustered_3.csv",
    road_points=road_points,
    n_drones={0: 2, 1: 2, 2: 2}
)

routes = routing_ts.optimize_all_clusters()

routing_ts.print_cluster_routes(routes)
routing_ts.visualize_cluster_routes(routes)



=== DRONE ROUTES (Tabu Search) ===
Cluster 0, Drone 1, Route: [0, 3, 4, 2, 25, 24, 26, 27, 28, 29, 30, 0]
Cluster 0, Drone 2, Route: [0, 1, 0]
Cluster 1, Drone 1, Route: [1, 15, 12, 59, 18, 20, 19, 60, 47, 62, 63, 61, 23, 22, 21, 53, 52, 48, 51, 50, 71, 72, 1]
Cluster 1, Drone 2, Route: [1, 49, 10, 9, 7, 55, 54, 6, 8, 56, 5, 11, 13, 14, 16, 17, 57, 58, 1]
Cluster 2, Drone 1, Route: [2, 31, 0, 36, 37, 38, 2]
Cluster 2, Drone 2, Route: [2, 66, 64, 65, 46, 67, 33, 34, 32, 42, 43, 41, 44, 39, 40, 35, 45, 68, 69, 70, 2]

Cluster 0 (Drones: 2)
  Drone 1 Route: R0 -> H3 -> H4 -> H2 -> H25 -> H24 -> H26 -> H27 -> H28 -> H29 -> H30 -> R0
    Total distance: 141.25 km | Est. time: 78.47 min
  Drone 2 Route: R0 -> H1 -> R0
    Total distance: 109.19 km | Est. time: 60.66 min

Cluster 1 (Drones: 2)
  Drone 1 Route: R1 -> H15 -> H12 -> H59 -> H18 -> H20 -> H19 -> H60 -> H47 -> H62 -> H63 -> H61 -> H23 -> H22 -> H21 -> H53 -> H52 -> H48 -> H51 -> H50 -> H71 -> H72 -> R1
    Total distance: 301.29 k

Tabu Search Bayesian Optimization

In [2]:
def objective(trial):
    tabu_tenure = trial.suggest_int("tabu_tenure", 5, 30)
    max_iter = trial.suggest_int("max_iter", 100, 500)
    neighborhood_size = trial.suggest_int("neighborhood_size", 1, 200)

    routing_ts = ClusterBasedDroneRoutingTS(
        csv_file="ketapang-12-08-24_merged_clustered_3.csv",
        road_points=road_points,
        n_drones={0: 2, 1: 2, 2: 2}
    )

    routes = routing_ts.optimize_all_clusters(
        tabu_tenure=tabu_tenure,
        max_iter=max_iter,
        neighborhood_size=neighborhood_size
    )

    # Hitung total distance dari semua cluster (km)
    total_distance = 0
    for cid, cluster_routes in routes.items():
        for route, hotspot_indices in cluster_routes:
            locs = [routing_ts.road_points[cid]] + [routing_ts.coordinates[i] for i in hotspot_indices]
            dist_matrix = routing_ts.build_dist_matrix(locs)
            for i in range(len(route)-1):
                total_distance += dist_matrix[route[i]][route[i+1]]

    return total_distance


In [3]:
import optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

print("Best params:", study.best_params)
print("Best total distance:", study.best_value)

[I 2025-10-10 20:29:33,046] A new study created in memory with name: no-name-47663b63-bc0a-475d-aa27-0fa9608cb954
[I 2025-10-10 20:29:33,967] Trial 0 finished with value: 965.8779015219114 and parameters: {'tabu_tenure': 20, 'max_iter': 100, 'neighborhood_size': 106}. Best is trial 0 with value: 965.8779015219114.
[I 2025-10-10 20:29:34,339] Trial 1 finished with value: 945.6409834560618 and parameters: {'tabu_tenure': 9, 'max_iter': 137, 'neighborhood_size': 18}. Best is trial 1 with value: 945.6409834560618.
[I 2025-10-10 20:29:35,316] Trial 2 finished with value: 944.1857200154361 and parameters: {'tabu_tenure': 26, 'max_iter': 111, 'neighborhood_size': 110}. Best is trial 2 with value: 944.1857200154361.
[I 2025-10-10 20:29:36,838] Trial 3 finished with value: 944.1857200154361 and parameters: {'tabu_tenure': 18, 'max_iter': 137, 'neighborhood_size': 189}. Best is trial 2 with value: 944.1857200154361.
[I 2025-10-10 20:29:38,157] Trial 4 finished with value: 944.1857200154361 and p

Best params: {'tabu_tenure': 26, 'max_iter': 111, 'neighborhood_size': 110}
Best total distance: 944.1857200154361


In [4]:
# best_params = {'tabu_tenure': 16, 'max_iter': 131, 'neighborhood_size': 70}
from ts import ClusterBasedDroneRoutingTS

road_points = {
    0: (-1.064124, 110.685258),   
    1: (-1.919500, 110.671412),  
    2: (-0.625036, 110.427354)    
}

# buat routing problem
routing_ts_opt = ClusterBasedDroneRoutingTS(
    csv_file="ketapang-12-08-24_merged_clustered_3.csv",
    road_points=road_points,
    n_drones={0: 2, 1: 2, 2: 2}
)

print("Running TS with params:", study.best_params)

routes_opt = routing_ts_opt.optimize_all_clusters(
    tabu_tenure=study.best_params["tabu_tenure"],
    max_iter=study.best_params["max_iter"],
    neighborhood_size=study.best_params["neighborhood_size"]
)

# tampilkan hasil
routing_ts.print_cluster_routes(routes_opt)
routing_ts.visualize_cluster_routes(routes_opt)


Running TS with params: {'tabu_tenure': 26, 'max_iter': 111, 'neighborhood_size': 110}

=== DRONE ROUTES (Tabu Search) ===
Cluster 0, Drone 1, Route: [0, 3, 4, 2, 25, 24, 26, 27, 28, 29, 30, 0]
Cluster 0, Drone 2, Route: [0, 1, 0]
Cluster 1, Drone 1, Route: [1, 15, 12, 59, 18, 20, 19, 60, 47, 62, 63, 61, 23, 22, 21, 53, 52, 48, 51, 50, 71, 72, 1]
Cluster 1, Drone 2, Route: [1, 49, 10, 9, 7, 55, 54, 6, 8, 56, 5, 11, 13, 14, 16, 17, 57, 58, 1]
Cluster 2, Drone 1, Route: [2, 31, 0, 36, 37, 38, 2]
Cluster 2, Drone 2, Route: [2, 66, 64, 65, 46, 67, 33, 34, 32, 42, 43, 41, 44, 39, 40, 35, 45, 68, 69, 70, 2]

Cluster 0 (Drones: 2)
  Drone 1 Route: R0 -> H3 -> H4 -> H2 -> H25 -> H24 -> H26 -> H27 -> H28 -> H29 -> H30 -> R0
    Total distance: 141.25 km | Est. time: 78.47 min
  Drone 2 Route: R0 -> H1 -> R0
    Total distance: 109.19 km | Est. time: 60.66 min

Cluster 1 (Drones: 2)
  Drone 1 Route: R1 -> H15 -> H12 -> H59 -> H18 -> H20 -> H19 -> H60 -> H47 -> H62 -> H63 -> H61 -> H23 -> H22 -> 

Tabu Search Melawi

In [1]:
from ts import *

road_points = {
    0: (-0.409011175, 112.1958254),   # Cluster 1
    1: (-0.592699463, 111.5859392),   # Cluster 2
    2: (-0.343085986, 111.7502586),   # Cluster 3 (Depo Utama BPBD)
    3: (-0.569912205, 111.9740824)    # Cluster 4
}

n_drones = {0: 2, 1: 2, 2: 2, 3: 2}

routing_ts = ClusterBasedDroneRoutingTSMelawi(
    csv_file="hotspot_clustered.csv",
    road_points=road_points,
    n_drones=n_drones
)

routes = routing_ts.optimize_all_clusters()
routing_ts.print_cluster_routes(routes)
routing_ts.visualize_cluster_routes(routes)


⚙️ Cluster numbering detected as 1-based → converting to 0-based internally.

=== DRONE ROUTES (Tabu Search, Fixed Version) ===

Cluster 1 (Drones: 2)
  Drone 1 Route: R1 -> H48 -> H9 -> H10 -> H6 -> H50 -> H51 -> R1
    Total distance: 76.11 km | Est. time: 42.28 min
  Drone 2 Route: R1 -> H7 -> H11 -> H8 -> H34 -> H52 -> R1
    Total distance: 49.62 km | Est. time: 27.56 min

Cluster 2 (Drones: 2)
  Drone 1 Route: R2 -> H39 -> H41 -> H40 -> H72 -> H69 -> H68 -> H22 -> H23 -> H70 -> H71 -> H24 -> H73 -> H74 -> H75 -> H76 -> R2
    Total distance: 62.89 km | Est. time: 34.94 min
  Drone 2 Route: R2 -> H65 -> H36 -> H64 -> H60 -> H62 -> H63 -> H61 -> H17 -> H18 -> H38 -> H20 -> H21 -> H19 -> H66 -> H67 -> R2
    Total distance: 96.18 km | Est. time: 53.43 min

Cluster 3 (Drones: 2)
  Drone 1 Route: R3 -> H1 -> H0 -> H2 -> H3 -> H4 -> R3
    Total distance: 48.45 km | Est. time: 26.92 min
  Drone 2 Route: R3 -> H54 -> H15 -> H16 -> H55 -> H56 -> R3
    Total distance: 54.07 km | Est. tim

Tabu Search Melawi Optimization

In [2]:
def objective_melawi(trial):
    # --- Hyperparameter search space ---
    tabu_tenure = trial.suggest_int("tabu_tenure", 5, 30)
    max_iter = trial.suggest_int("max_iter", 100, 500)
    neighborhood_size = trial.suggest_int("neighborhood_size", 5, 200)

    # --- Konfigurasi titik depot (road_points) untuk dataset Melawi ---
    road_points = {
        0: (-0.409011175, 112.1958254),   # Cluster 1
        1: (-0.592699463, 111.5859392),   # Cluster 2
        2: (-0.343085986, 111.7502586),   # Cluster 3 (Depo Utama BPBD)
        3: (-0.569912205, 111.9740824)    # Cluster 4
    }

    n_drones = {0: 2, 1: 2, 2: 2, 3: 2}

    # --- Inisialisasi class untuk Melawi ---
    routing_ts = ClusterBasedDroneRoutingTSMelawi(
        csv_file="hotspot_clustered.csv",
        road_points=road_points,
        n_drones=n_drones
    )

    # --- Jalankan optimasi Tabu Search untuk semua cluster ---
    routes = routing_ts.optimize_all_clusters(
        tabu_tenure=tabu_tenure,
        max_iter=max_iter,
        neighborhood_size=neighborhood_size
    )

    # --- Hitung total jarak dari semua cluster ---
    total_distance = 0
    for cid, cluster_routes in routes.items():
        for route, hotspot_indices in cluster_routes:
            locs = [routing_ts.road_points[cid]] + [routing_ts.coordinates[i] for i in hotspot_indices]
            dist_matrix = routing_ts.build_dist_matrix(locs)
            for i in range(len(route)-1):
                total_distance += dist_matrix[route[i]][route[i+1]]

    return total_distance


In [3]:
import optuna

# --- Buat study baru untuk dataset Melawi ---
study_melawi = optuna.create_study(
    direction="minimize",
    study_name="melawi_tabu_search_optimization"
)

# --- Jalankan optimasi ---
study_melawi.optimize(objective_melawi, n_trials=10)

# --- Cetak hasil terbaik ---
print("\n🏆 Best parameters (Melawi):", study_melawi.best_params)
print(f"🚀 Best total distance (Melawi): {study_melawi.best_value:.2f} km")


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-10-12 15:21:57,651] A new study created in memory with name: melawi_tabu_search_optimization


⚙️ Cluster numbering detected as 1-based → converting to 0-based internally.


[I 2025-10-12 15:21:58,187] Trial 0 finished with value: 511.75881676445704 and parameters: {'tabu_tenure': 29, 'max_iter': 113, 'neighborhood_size': 34}. Best is trial 0 with value: 511.75881676445704.


⚙️ Cluster numbering detected as 1-based → converting to 0-based internally.


[I 2025-10-12 15:21:59,081] Trial 1 finished with value: 511.9204272477779 and parameters: {'tabu_tenure': 5, 'max_iter': 118, 'neighborhood_size': 85}. Best is trial 0 with value: 511.75881676445704.


...

⚙️ Cluster numbering detected as 1-based → converting to 0-based internally.


[I 2025-10-12 15:22:08,896] Trial 8 finished with value: 511.0145741523468 and parameters: {'tabu_tenure': 19, 'max_iter': 108, 'neighborhood_size': 75}. Best is trial 2 with value: 511.0145741523468.


⚙️ Cluster numbering detected as 1-based → converting to 0-based internally.


[I 2025-10-12 15:22:12,134] Trial 9 finished with value: 511.75881676445704 and parameters: {'tabu_tenure': 19, 'max_iter': 488, 'neighborhood_size': 153}. Best is trial 2 with value: 511.0145741523468.



🏆 Best parameters (Melawi): {'tabu_tenure': 29, 'max_iter': 259, 'neighborhood_size': 173}
🚀 Best total distance (Melawi): 511.01 km


In [4]:
best_params = study_melawi.best_params

# Titik depot (road_points) khusus Melawi
road_points = {
    0: (-0.409011175, 112.1958254),   # Cluster 1
    1: (-0.592699463, 111.5859392),   # Cluster 2
    2: (-0.343085986, 111.7502586),   # Cluster 3 (Depo Utama BPBD)
    3: (-0.569912205, 111.9740824)    # Cluster 4
}

# Jumlah drone tiap cluster
n_drones = {0: 2, 1: 2, 2: 2, 3: 2}

# Inisialisasi routing problem dengan dataset Melawi
routing_ts_opt_melawi = ClusterBasedDroneRoutingTSMelawi(
    csv_file="hotspot_clustered.csv",
    road_points=road_points,
    n_drones=n_drones
)

print("\n🚀 Running Tabu Search for Melawi with best Optuna params:")
print(best_params)

# Jalankan Tabu Search dengan parameter terbaik
routes_opt_melawi = routing_ts_opt_melawi.optimize_all_clusters(
    tabu_tenure=best_params["tabu_tenure"],
    max_iter=best_params["max_iter"],
    neighborhood_size=best_params["neighborhood_size"]
)

routing_ts_opt_melawi.print_cluster_routes(routes_opt_melawi)
routing_ts_opt_melawi.visualize_cluster_routes(routes_opt_melawi)


⚙️ Cluster numbering detected as 1-based → converting to 0-based internally.

🚀 Running Tabu Search for Melawi with best Optuna params:
{'tabu_tenure': 29, 'max_iter': 259, 'neighborhood_size': 173}

=== DRONE ROUTES (Tabu Search, Fixed Version) ===

Cluster 1 (Drones: 2)
  Drone 1 Route: R1 -> H48 -> H9 -> H10 -> H6 -> H50 -> H51 -> R1
    Total distance: 76.11 km | Est. time: 42.28 min
  Drone 2 Route: R1 -> H7 -> H11 -> H8 -> H34 -> H52 -> R1
    Total distance: 49.62 km | Est. time: 27.56 min

Cluster 2 (Drones: 2)
  Drone 1 Route: R2 -> H39 -> H41 -> H40 -> H72 -> H69 -> H68 -> H22 -> H23 -> H70 -> H71 -> H24 -> H73 -> H74 -> H75 -> H76 -> R2
    Total distance: 62.89 km | Est. time: 34.94 min
  Drone 2 Route: R2 -> H65 -> H36 -> H64 -> H60 -> H62 -> H63 -> H61 -> H17 -> H18 -> H38 -> H20 -> H21 -> H19 -> H66 -> H67 -> R2
    Total distance: 96.18 km | Est. time: 53.43 min

Cluster 3 (Drones: 2)
  Drone 1 Route: R3 -> H1 -> H0 -> H2 -> H3 -> H4 -> R3
    Total distance: 48.45 km |